In [3]:
import sys
import os as os
import numpy as np
import polars as pl
import pandas as pd
import geopandas as gpd
import json
import math
import joblib
import copy
import sqlite3
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from pygam import LinearGAM, s
from sklearn.metrics import mean_absolute_percentage_error

from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import unary_union
# from dataclasses import dataclass, field
# from typing_extensions import List, Dict, Tuple

if 'scicore' in os.getcwd():
    path = '/scicore/home/krysiak/hocrau00/ondemand/OptimalPV_RH'
else:
    path = os.getcwd().split('\\src')[0]
    %load_ext rpy2.ipython

os.chdir(path)
from src.calibration_class import Calibration_Settings, Calibration



Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


 * Finished random forest regression approach 2 * 


## explore "economic pvinstallation" 
house holds with e.g. 20% < pvroof-rato < 70% 

In [ ]:
calib_sett = Calibration_Settings(
            name_dir_export='calib_all_CH_bfs3',
            name_calib_subscen='reg2_all_CH_bfs',
            kt_numbers=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,], 
            # bfs_numbers=[5391, 5394, 
            #              120, 4726, 4761, 4651, 
            #              4203, 4204,
            #              ],
            # concat_bfs_subsample              = True, 

            n_rows_import                     = None,
            rerun_import_and_preprp_data_TF   = True,
            export_gwr_ALL_building_gdf_TF    = False, 

            run_concatenate_preprep_data_TF         = True,
            run_approach1_fit_optim_costfunction_TF = True, 
            run_approach2_regression_instsize_TF    = True,
            run_appr2_random_forest_reg_TF          = True,

            reg2_random_forest_reg_settings         = {
                'run_ML_rfr_TF': False,
                'visualize_ML_rfr_TF': True,
                'reg2_rfrname_dfsuffix_dicts': {
                # #     'mod_GRIDSEARCH_CONVOLUTION_2': {
                # #         'rfr_mod_name': '_rfr2',
                # #         'df_suffix': '_kwpmax20',
                # #         'random_state':         None,    # default: None  # | None,
                # #         'n_jobs':               -1,      # default: None  # | -1,
                # #         'cross_validation':     None,
                # #         'n_estimators':         [1, ]  ,    # default: 100   # | 1,
                # #         'min_samples_split':    [5, ]    ,    # default: 2     # | 1000,
                # #         'max_depth':            [3, ]   ,    # default: None  # | 3,
                # # },
                    'mod1a': {
                        'rfr_mod_name': '_rfr1', 
                        'df_suffix': '',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         400  ,    # default: 100   # | 1,       
                        'min_samples_split':    2    ,    # default: 2     # | 1000,    
                        'max_depth':            40   ,    # default: None  # | 3,       
                        'kWp_segments': [(None, None)],

                }, 
                    'mod1b': {
                        'rfr_mod_name': '_rfr1', 
                        'df_suffix': '',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         150  ,    # default: 100   # | 1,       
                        'min_samples_split':    50    ,    # default: 2     # | 1000,    
                        'max_depth':            10   ,    # default: None  # | 3,       
                        'kWp_segments': [(None, None)],

                }, 
                    'mod2': {
                        'rfr_mod_name': '_rfr2', 
                        'df_suffix': '_pvroof20to70',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         400  ,    # default: 100   # | 1,       
                        'min_samples_split':    2     ,    # default: 2     # | 1000,    
                        'max_depth':            30   ,    # default: None  # | 3,
                        'kWp_segments': [(None, None)],
       
                }, 
                    'mod3': {
                        'rfr_mod_name': '_rfr3', 
                        'df_suffix': '_res1to2',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         400  ,    # default: 100   # | 1,       
                        'min_samples_split':    2    ,    # default: 2     # | 1000,    
                        'max_depth':            40   ,    # default: None  # | 3,       
                        'kWp_segments': [(None, None)],
                }, 
                    'mod4': {
                        'rfr_mod_name': '_rfr4', 
                        'df_suffix': '',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         20  ,    # default: 100   # | 1,       
                        'min_samples_split':    2     ,    # default: 2     # | 1000,    
                        'max_depth':            30   ,    # default: None  # | 3,       
                        'kWp_segments': [
                            (0, 5), 
                            (5, 7.5),
                            (7.5, 10),
                            (10, 12.5), 
                            (12.5, 17.5),
                            (17.5, 25),
                            (25, 100), 
                        ], 
                }, 
                    'mod5': {
                        'rfr_mod_name': '_rfr5', 
                        'df_suffix': '_pvroof20to70',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         20  ,    # default: 100   # | 1,       
                        'min_samples_split':    2     ,    # default: 2     # | 1000,    
                        'max_depth':            30   ,    # default: None  # | 3,       
                        'kWp_segments': [
                            (0, 5), 
                            (5, 7.5),
                            (7.5, 10),
                            (10, 12.5), 
                            (12.5, 17.5),
                            (17.5, 25),
                            (25, 100), 
                        ], 
                }, 
                'mod6': {
                        'rfr_mod_name': '_rfr6', 
                        'df_suffix': '',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         20  ,    # default: 100   # | 1,       
                        'min_samples_split':    2     ,    # default: 2     # | 1000,    
                        'max_depth':            30   ,    # default: None  # | 3,       
                        'kWp_segments': [
                            ( 0.00,  6.00), 
                            ( 6.00,  8.00),
                            ( 8.00,  9.00),
                            ( 9.00, 10.00),
                            (10.00, 11.00), 
                            (11.00, 12.00),
                            (12.00, 14.00),
                            (14.00, 16.00),
                            (16.00, 20.00),
                            (20.00, 99.00)
                        ], 
                }, 
                    'mod7': {
                        'rfr_mod_name': '_rfr7', 
                        'df_suffix': '_pvroof20to70',

                        'random_state':         24,    # default: None  # | None,    
                        'n_jobs':               -1,      # default: None  # | -1,  
                        'cross_validation':     None, 
                        'n_estimators':         20  ,    # default: 100   # | 1,       
                        'min_samples_split':    2     ,    # default: 2     # | 1000,    
                        'max_depth':            30   ,    # default: None  # | 3,       
                        'kWp_segments': [
                            ( 0.00,  6.00), 
                            ( 6.00,  8.00),
                            ( 8.00,  9.00),
                            ( 9.00, 10.00),
                            (10.00, 11.00), 
                            (11.00, 12.00),
                            (12.00, 14.00),
                            (14.00, 16.00),
                            (16.00, 20.00),
                            (20.00, 99.00)
                        ], 
                }, 


            }},
        )
calib_class = Calibration(calib_sett)

calib_class.random_forest_regression()


  START CALIBRATION: calib_all_CH_bfs3
  > name_preprep_subsen: preprep_class_default_sett
  > name_calib_subscen: reg2_all_CH_bfs



## explore missing pvs in reg2
find out more about (huge) amount of missing pv inst in `reg2_all_CH_bfs_df_approach2`

In [6]:
# COMPARISON PV IN ESTIMATING SAMPLE 

# gm_shp_gdf
# gm_shp_gdf = gpd.read_file(f'{self.sett.data_path}/input/swissboundaries3d_2023-01_2056_5728.shp', layer ='swissBOUNDARIES3D_1_4_TLM_HOHEITSGEBIET')
gm_shp_gdf = gpd.read_file(R"C:\Models\OptimalPV_RH\data\input\swissboundaries3d_2023-01_2056_5728.shp\swissBOUNDARIES3D_1_4_TLM_HOHEITSGEBIET.shp", layer ='swissBOUNDARIES3D_1_4_TLM_HOHEITSGEBIET')
gm_shp_gdf = gm_shp_gdf.drop(columns = ['DATUM_AEND', 'DATUM_ERST'])
gm_shp_gdf = gm_shp_gdf.loc[gm_shp_gdf['KANTONSNUM'].notna()]
gm_shp_gdf['KANTONSNUM'] = gm_shp_gdf['KANTONSNUM'].astype(int)
gm_shp_gdf['BFS_NUMMER'] = gm_shp_gdf['BFS_NUMMER'].astype(str)
gm_shp_gdf.crs = 'EPSG:2056'

def attach_bfs_to_spatial_data(gdf, gm_shp_df, keep_cols = ['BFS_NUMMER', 'geometry' ]):
    """
    Function to attach BFS numbers to spatial data sources
    """
    gdf = copy.deepcopy(gdf)
    gdf.set_crs(gm_shp_df.crs, allow_override=True, inplace=True)
    gdf = gpd.sjoin(gdf, gm_shp_df, how="left", predicate="within")
    dele_cols = ['index_right'] + [col for col in gm_shp_df.columns if col not in keep_cols]
    gdf.drop(columns = dele_cols, inplace = True)
    if 'BFS_NUMMER' in gdf.columns:
        # transform BFS_NUMMER to str, np.nan to ''
        gdf['BFS_NUMMER'] = gdf['BFS_NUMMER'].apply(lambda x: '' if pd.isna(x) else str(int(x)))

    return gdf


# import rfr 
rfr_pl = pl.read_csv(r"C:\Models\OptimalPV_RH\data\calibration\PVALLOC_calibration_model_coefs\reg2_all_CH_bfs_df_approach2.csv")
rfr_pl = rfr_pl.with_columns([
    pl.col('xtf_id').cast(pl.Utf8),
    pl.col('BFS_NUMMER').cast(pl.Utf8),
    pl.col('GKLAS').cast(pl.Utf8),
    pl.col('GSTAT').cast(pl.Utf8),
    pl.col('GWAERZH1').cast(pl.Utf8),
])


# import pv
elec_prod_gdf = gpd.read_file(
    r"C:\Models\OptimalPV_RH\data\input\ch.bfe.elektrizitaetsproduktionsanlagen_gpkg\ch.bfe.elektrizitaetsproduktionsanlagen.gpkg",
    layer ='ElectricityProductionPlant', 
    )
pv_all_gdf = elec_prod_gdf.loc[elec_prod_gdf['SubCategory'] == 'subcat_2'].copy()
pv_all_gdf['xtf_id'] = pv_all_gdf['xtf_id'].astype(str)
pv_all_gdf = attach_bfs_to_spatial_data(pv_all_gdf, gm_shp_gdf)
pv_all_pl = pl.DataFrame(pv_all_gdf.drop(columns='geometry'))


# import gwr
query_columns = ['EGID', 'GDEKT', 'GGDENR', 'GKODE', 'GKODN', 'GKSCE', 'GSTAT', 'GKAT', 'GKLAS', 'GBAUJ', 'GBAUM', 'GBAUP', 'GABBJ', 'GANZWHG','GWAERZH1', 'GENH1', 'GWAERSCEH1', 'GWAERDATH1', 'GEBF', 'GAREA']
query_columns_str = ', '.join(query_columns)

conn = sqlite3.connect(r"C:\Models\OptimalPV_RH\data\input/GebWohnRegister.CH/data.sqlite")
cur = conn.cursor()
cur.execute(f'SELECT {query_columns_str} FROM building')
sqlrows = cur.fetchall()
conn.close()

gwr_all_building_df = pd.DataFrame(sqlrows, columns=query_columns)

def gwr_to_gdf(df):
    df = copy.deepcopy(df)                
    df = df.loc[(df['GKODE'] != '') & (df['GKODN'] != '')]
    df[['GKODE', 'GKODN']] = df[['GKODE', 'GKODN']].astype(float)
    df['geometry'] = df.apply(lambda row: Point(row['GKODE'], row['GKODN']), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.crs = 'EPSG:2056'
    return gdf

gwr_all_building_gdf = gwr_to_gdf(gwr_all_building_df)


# interset gwr + pv
def set_crs_to_gm_shp(gdf_CRS, gdf_a, gdf_b = None):
    gdf_a.set_crs(gdf_CRS.crs, allow_override=True, inplace=True)
    if gdf_b is not None:
        gdf_b.set_crs(gdf_CRS.crs, allow_override=True, inplace=True)
    
    if gdf_b is None: 
        return gdf_a
    if gdf_b is not None:
        return gdf_a, gdf_b


gwr_all_building_buff = gwr_all_building_gdf
gwr_buff_gdf = copy.deepcopy(gwr_all_building_buff)
gwr_buff_gdf.set_crs("EPSG:32632", allow_override=True, inplace=True)
gwr_buff_gdf['geometry'] = gwr_buff_gdf['geometry'].buffer(10)
gwr_buff_gdf, pv_all_gdf = set_crs_to_gm_shp(gm_shp_gdf, gwr_buff_gdf, pv_all_gdf)

gwregid_pvid_join = gpd.sjoin(pv_all_gdf,gwr_buff_gdf, how="left", predicate="within")
gwregid_pvid_join.drop(columns = ['index_right'] + [col for col in gwr_buff_gdf.columns if col not in ['EGID', 'geometry']], inplace = True)

# keep only unique xtf_ids 
gwregid_pvid_unique = copy.deepcopy(gwregid_pvid_join.loc[~gwregid_pvid_join.duplicated(subset='xtf_id', keep=False)])
xtf_duplicates =      copy.deepcopy(gwregid_pvid_join.loc[ gwregid_pvid_join.duplicated(subset='xtf_id', keep=False)])

if xtf_duplicates.shape[0] > 0:
    # match duplicates with nearest neighbour
    xtf_nearestmatch_list = []
    # xtf_id = xtf_duplicates['xtf_id'].unique()[0]
    for xtf_id in xtf_duplicates['xtf_id'].unique():
        gwr_sub = copy.deepcopy(gwr_buff_gdf.loc[gwr_buff_gdf['EGID'].isin(xtf_duplicates.loc[xtf_duplicates['xtf_id'] == xtf_id, 'EGID'])])
        pv_sub = copy.deepcopy(pv_all_gdf.loc[pv_all_gdf['xtf_id'] == xtf_id])
        
        assert pv_sub.crs == gwr_sub.crs
        gwr_sub['distance_to_pv'] = gwr_sub['geometry'].centroid.distance(pv_sub['geometry'].values[0])
        pv_sub['EGID'] = gwr_sub.loc[gwr_sub['distance_to_pv'].idxmin()]['EGID']

        xtf_nearestmatch_list.append(pv_sub)
    
    xtf_nearestmatches_df = pd.concat(xtf_nearestmatch_list, ignore_index=True)
    gwregid_pvid = pd.concat([gwregid_pvid_unique, xtf_nearestmatches_df], ignore_index=True).drop_duplicates()

else: 
    gwregid_pvid = gwregid_pvid_unique

os.makedirs(r"C:\Models\OptimalPV_RH\data\calibration\calibration_check ", exist_ok=True)
with open(r"C:\Models\OptimalPV_RH\data\calibration\calibration_check\gwregid_pvid.geojson", 'w') as f:
    f.write(gwregid_pvid.to_json())
    


In [7]:
# print('pv_all_pl')
# for col in pv_all_pl.columns:
#     print(f'{col:25} type: {pv_all_pl[col].dtype}')

# print('\nrfr_pl')
# for col in rfr_pl.columns:
#     print(f'{col:25} type: {rfr_pl[col].dtype}')

pv_not_rfr = pv_all_pl.filter(~pl.col('xtf_id').is_in(rfr_pl['xtf_id'].implode()))

print(f'n_pv not in calibration: {pv_not_rfr.shape[0]/pv_all_pl.shape[0]*100:.1f}% -> {pv_not_rfr.shape[0]} not in rfr of {pv_all_pl.shape[0]} pv_all_pl installations')
bfs_list, n_pv_bfs_list, n_pv_not_rfr_bfs_list, share_pv_not_rfr_list = [], [], [], []
for bfs_num in gm_shp_gdf['BFS_NUMMER'].unique():
    n_pv_bfs = pv_all_pl.filter(pl.col('BFS_NUMMER') == bfs_num).shape[0]
    n_pv_not_rfr_bfs = pv_not_rfr.filter(pl.col('BFS_NUMMER') == bfs_num).shape[0]

    bfs_list.append(bfs_num)
    n_pv_bfs_list.append(n_pv_bfs)
    n_pv_not_rfr_bfs_list.append(n_pv_not_rfr_bfs)
    share_pv_not_rfr_list.append(n_pv_not_rfr_bfs / n_pv_bfs if n_pv_bfs > 0 else 0)

bfs_pv_nan_df = pd.DataFrame({
    'BFS_NUMMER': bfs_list,
    'n_pv_bfs': n_pv_bfs_list,
    'n_pv_not_rfr_bfs': n_pv_not_rfr_bfs_list,
    'share_pv_not_rfr': share_pv_not_rfr_list,
    })
print(f'n_bfs completely left out (n_pv in gm == n_pv not in rfr): {bfs_pv_nan_df.loc[bfs_pv_nan_df["n_pv_bfs"] == bfs_pv_nan_df["n_pv_not_rfr_bfs"]].shape[0]/bfs_pv_nan_df.shape[0]*100:.1f}% -> {bfs_pv_nan_df.loc[bfs_pv_nan_df["n_pv_bfs"] == bfs_pv_nan_df["n_pv_not_rfr_bfs"]].shape[0]} of {bfs_pv_nan_df.shape[0]} bfs areas')



n_pv not in calibration: 93.1% -> 222661 not in rfr of 239214 pv_all_pl installations
n_bfs completely left out (n_pv in gm == n_pv not in rfr): 58.1% -> 1249 of 2149 bfs areas


In [8]:
# charactersitics of pv in bfs which are not completely omitted
bfs_complete_nan = bfs_pv_nan_df.loc[bfs_pv_nan_df['share_pv_not_rfr'] == 1.0, 'BFS_NUMMER'].tolist()
pv2 = pv_all_pl.filter( 
    (~pl.col('BFS_NUMMER').is_in(bfs_complete_nan)) & 
    (~pl.col('xtf_id').is_in(rfr_pl['xtf_id'].implode()))
    )
pv2.shape
pv2.head()

# print(f'n_pv not in calibration (only bfs with at least one pv in rfr): 
# bfs_pv_nan_df = pl.DataFrame(bfs_pv_nan_dict).transpose().with_columns([
#     pl.col('n_pv_bfs').cast(pl.Int64),
#     pl.col('n_pv_not_rfr_bfs').cast(pl.Int64),
#     pl.col('share_pv_not_rfr').cast(pl.Float64),    
# ])


xtf_id,Address,PostCode,Municipality,Canton,BeginningOfOperation,InitialPower,TotalPower,MainCategory,SubCategory,PlantCategory,BFS_NUMMER
str,str,i32,str,str,str,f32,f32,str,str,str,str
"""10797""","""Brüggelbachstrasse 9""",3176,"""Neuenegg""","""BE""","""2008-11-20""",12.5,12.5,"""maincat_2""","""subcat_2""","""plantcat_9""","""670"""
"""11795""","""Via Industria 1""",6934,"""Bioggio""","""TI""","""2009-08-17""",124.199997,1541.079956,"""maincat_2""","""subcat_2""","""plantcat_8""","""5151"""
"""14763""","""Kürzematt 12""",4515,"""Oberdorf SO""","""SO""","""2009-12-07""",6.3,6.3,"""maincat_2""","""subcat_2""","""plantcat_8""",""""""
"""14764""","""Winkel""",4234,"""Zullwil""","""SO""","""2008-11-07""",3.2,3.2,"""maincat_2""","""subcat_2""","""plantcat_8""",""""""
"""14775""","""Katzacker 9""",3368,"""Bleienbach""","""BE""","""2009-11-25""",4.2,4.2,"""maincat_2""","""subcat_2""","""plantcat_8""","""324"""
